In [ ]:
!pip install validator-collection

In [1]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import os
import urllib.request, urllib.error
from validator_collection import validators, checkers

In [2]:
project_path = '/home/mgarcia/Documentos/ironhack/ejercicios/module-1/pandas-project/your-code/' 

In [ ]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files('teajay/global-shark-attacks')
file = 'global-shark-attacks.zip'
with zipfile.ZipFile(project_path + file, 'r') as zip_ref:
    zip_ref.extractall(project_path)

In [22]:
raw = pd.read_csv(project_path + 'attacks.csv', encoding = 'latin_1')
raw.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


### Hypothesis:
1. Most attacks were unprovoked
2. Most victims were males between 20 and 40 years
3. Most victims were surfing

## Cleaning empty rows

In [23]:
print(raw.shape)
raw.isnull().sum()

(25723, 24)


Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [24]:
# if case number is null, maybe all the row is null. Let's check
raw_clean=raw[~raw['Case Number'].isnull()]
print(raw_clean.shape)
raw_clean.isnull().sum()

# indeed :D

(8702, 24)


Case Number                  0
Date                      2401
Year                      2403
Type                      2405
Country                   2451
Area                      2856
Location                  2941
Activity                  2945
Name                      2611
Sex                       2966
Age                       5231
Injury                    2429
Fatal (Y/N)               2940
Time                      5754
Species                   5238
Investigator or Source    2418
pdf                       2401
href formula              2402
href                      2401
Case Number.1             2401
Case Number.2             2401
original order            2394
Unnamed: 22               8701
Unnamed: 23               8700
dtype: int64

In [25]:
# Also, seems like null dates mean empty row
raw_clean=raw[~raw['Date'].isnull()]
print(raw_clean.shape)
raw_clean.isnull().sum()

# confirm

(6302, 24)


Case Number                  1
Date                         0
Year                         2
Type                         4
Country                     50
Area                       455
Location                   540
Activity                   544
Name                       210
Sex                        565
Age                       2831
Injury                      28
Fatal (Y/N)                539
Time                      3354
Species                   2838
Investigator or Source      17
pdf                          0
href formula                 1
href                         0
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               6301
Unnamed: 23               6300
dtype: int64

In [26]:
# The dataset has reduce a good amount of data, so now it will be easier to inspect it

raw_clean.to_csv('attacks_clean.csv', ';', quotechar = '"')

 ## Dropping unnecesary columns


In [27]:
# two last columns are empty so we can drop them
#columns = raw.columns
raw_clean = raw_clean.drop(columns=['Unnamed: 22', 'Unnamed: 23'])

# As we're taking Case Number as reference, we don't need the duplicated Case Number columns
raw_clean = raw_clean.drop(columns=['Case Number.1', 'Case Number.2'])

# original order isn't useful for our purposes
raw_clean = raw_clean.drop(columns=['original order'])

In [28]:
# href and href formula looks pretty much the same (href formula has one extra null)
(raw_clean['href formula'] == raw_clean['href']).value_counts()

True     6242
False      60
dtype: int64

In [14]:
#The code below is quite heavy, I wouldn't run it unless I'm sure I have time. 
#The results were:
#requests_failed =0
#invalid_url = 73

#requests_failed = 0
#invalid_url = 0
#for url in raw_clean['href']:
#    if checkers.is_url(url) == True:
#        try:
#            resp = urllib.request.urlopen(url).getcode()
#        except urllib.error.HTTPError as e:
#            if e.code == 404:
#                requests_failed = requests_failed + 1
#    else:
#        invalid_url = invalid_url + 1

#print(requests_failed)
#print(invalid_url)

0
73


In [29]:
#So we can drop one of the columns without losing a lot of information as they match at 99%
raw_clean = raw_clean.drop(columns=['href formula'])

In [30]:
# pdf is just the name of the pdf file that stores the data, as long as we can donwload it. 

raw_clean = raw_clean.drop(columns=['pdf'])

## Deleting duplicates

In [31]:
# first check: full duplicate rows
print(sum(raw_clean.duplicated()))
raw_clean = raw_clean.drop_duplicates()

0


In [32]:
# second check: case numbers
print(raw_clean.duplicated(subset='Case Number').sum())

pd.concat(g for _, g in raw_clean.groupby('Case Number') if len(g) > 1)

#Can't drop this 
#raw_clean = raw_clean.drop_duplicates(subset='Case Number', keep = False)

16


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,href
5443,1907.10.16.R,Reported 16-Oct-1907,1907.0,Unprovoked,CHINA,Hong Kong,"Sharp Peak, Sai Kung Peninsula, New Territories",Fishing,fishermen,M,NaN,"3 of thel 5 were injured, one of whom lost bot...",N,NaN,Shark involvement probable,"Dawson Daily News, 11/20/1907",http://sharkattackfile.net/spreadsheets/pdf_di...
5444,1907.10.16.R,Reported 16-Oct-1907,1907.0,Unprovoked,CHINA,Hong Kong,"Sharp Peak, Sai Kung Peninsula, New Territories",Fishing,fishermen,M,NaN,2 of the 5 fishermen were so seriously injure...,Y,NaN,Shark involvement probable,"Dawson Daily News, 11/20/1907",http://sharkattackfile.net/spreadsheets/pdf_di...
5354,1913.08.27.R,Reported 27-Aug-1913,1913.0,Invalid,USA,New Jersey,"Lavalette, Ocean County",NaN,NaN,M,NaN,Man's leg recovered from 800-lb shark,NaN,NaN,Shark involvement prior to death unconfirmed,"Trenton Evening Times, 8/27/1913",http://sharkattackfile.net/spreadsheets/pdf_di...
5355,1913.08.27.R,Reported 27-Aug-1913,1913.0,Invalid,USA,New Jersey,"Spring Lake, Monmouth County",NaN,NaN,F,NaN,Female foot recovered from shark,NaN,NaN,Shark involvement prior to death unconfirmed,"Washington Post, 8/27/1913",http://sharkattackfile.net/spreadsheets/pdf_di...
5322,1915.07.06.a.R,Reported 06-Jul-1915,1915.0,Unprovoked,MEXICO,Tamaulipas,Tampico,Fishing,Captain Thaxton,M,NaN,FATAL,Y,NaN,NaN,"Oakland Tribune, 7/6/1915",http://sharkattackfile.net/spreadsheets/pdf_di...
5323,1915.07.06.a.R,Reported 06-Jul-1915,1915.0,Unprovoked,MEXICO,NaN,Santa Maria Bar,Wading,J.W. McDonald,M,NaN,FATAL,Y,NaN,NaN,"Oakland Tribune, 7/6/1915",http://sharkattackfile.net/spreadsheets/pdf_di...
5260,1920.00.00.b,1920s,1920.0,Unprovoked,JAMAICA,Westmoreland Parish,Savanna-la-Mar,Jumped overboard,sailor,M,NaN,FATAL,Y,NaN,NaN,"Daily Gleaner, 4/14/1969",http://sharkattackfile.net/spreadsheets/pdf_di...
5261,1920.00.00.b,1920,1920.0,Unprovoked,NEW ZEALAND,North Island,"Stanley Bay, Auckland Harbor",NaN,female,F,NaN,"""Recovered""",N,NaN,NaN,"V.M. Coppleson (1958), p.262",http://sharkattackfile.net/spreadsheets/pdf_di...
5213,1923.00.00.a,1923,1923.0,Provoked,USA,New Jersey,Ocean City (offshore),Hoisting shark aboard fishing boat,male,M,NaN,Shark's tail broke his leg. PROVOKED INCIDENT,N,NaN,NaN,"Ref in New York Herald Tribune, 8/23/1960; V.M...",http://sharkattackfile.net/spreadsheets/pdf_di...
5214,1923.00.00.a,1923,1922.0,Provoked,USA,New Jersey,Ocean City (offshore),Hoisting shark aboard fishing boat,male,M,NaN,Shark's tail broke his leg. PROVOKED INCIDENT,N,NaN,NaN,"Ref in New York Herald Tribune, 8/23/1960; V.M...",http://sharkattackfile.net/spreadsheets/pdf_di...


In [21]:
# third check: href. If it points to the same report, it probaby be the same case
print(raw_clean.duplicated(subset='href').sum())

pd.concat(g for _, g in raw_clean.groupby("href") if len(g) > 1)

#raw_clean[href.isin(ids[ids.duplicated()])].sort("ID")
#raw_clean = raw_clean.drop_duplicates(subset='href', keep = False)

15


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,href
5597,1898.00.00.R,Reported 1898,1898.0,Invalid,CRETE,NaN,NaN,Sponge divers,male,M,NaN,Unknown,NaN,NaN,Questionable,"C. Moore, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
5825,1880.00.00.e,1880?,1880.0,Invalid,SYRIA,NaN,NaN,Diving for sponges,male,M,NaN,FATAL,NaN,NaN,Shark involvement prior to death unconfirmed,"C. Moore, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
5469,1906.09.27.R.b,Reported 27-Sep-1906,1906.0,Unprovoked,INDONESIA,Java,Lorok Bay,Swimming,William Munich,M,NaN,Thumb & index finger of left hand severed,N,NaN,NaN,"Atlanta Constitution, 9/27/1906",http://sharkattackfile.net/spreadsheets/pdf_di...
5470,1906.09.27.R.a,Reported 27-Sep-1906,1906.0,Unprovoked,INDONESIA,Java,Lorok Bay,Swimming,a Swedish sailor,M,NaN,FATAL,Y,NaN,NaN,"Atlanta Constitution, 9/27/1906",http://sharkattackfile.net/spreadsheets/pdf_di...
5305,1916.07.12.b,12-Jul-1916,1916.0,Unprovoked,USA,New Jersey,"Matawan Creek, 10 miles from the sea, Monmouth...",Swimming (recovering remains of Stilwell),Stanley Fisher,M,24,"FATAL, thigh bitten",Y,NaN,3 m [10'] shark,"R. Fernicola, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
5306,1916.07.12.a,12-Jul-1916,1916.0,Unprovoked,USA,New Jersey,"Matawan Creek, 10 miles from the sea, Monmouth...",Swimming,Lester Stillwell,M,10,"FATAL, legs & torso bitten",Y,A.M.,NaN,"R. Fernicola, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
5294,1916.12.08.b,08-Dec-1916,1916.0,Unprovoked,AUSTRALIA,New South Wales,"Seven Shillings Beach, Middle Harbor, Sydney (...",Taking wife to beach & about 1 m from the shore,Walter C. German,M,41,"FATAL, right arm severed, chest punctured",Y,09h00,NaN,"G.P. Whitley, ref. Dr. Cleland, 1924; V.M. Cop...",http://sharkattackfile.net/spreadsheets/pdf_di...
5295,1916.12.08.a,08-Dec-1916,1916.0,Unprovoked,AUSTRALIA,New South Wales,"Seven Shillings Beach, Middle Harbor, Sydney (...",Swimming 10 m from shore,Mrs Walter German,F,NaN,Leg nipped by shark,N,08h58,NaN,"A. Sharpe, p.71",http://sharkattackfile.net/spreadsheets/pdf_di...
5237,1921.11.27.b,27-Nov-1921,1921.0,Unprovoked,AUSTRALIA,Queensland,"Gays Corner, Bulimba Reach of Brisbane River",Fell from his father's back into the water,George Jack,M,8,"FATAL, disappeared, body not recovered",Y,Morning,NaN,"V.M. Coppleson (1958), pp.92 & 237",http://sharkattackfile.net/spreadsheets/pdf_di...
5238,1921.11.27.a,27-Nov-1921,1921.0,Unprovoked,AUSTRALIA,Queensland,"Gays Corner, Bulimba Reach of Brisbane River","Wading to dinghy, carrying his son",Herbert Jack,M,40,"Right hip, buttock, elbow, arm & wrist bitten",N,Morning,NaN,V.M. Coppleson.Q3. (1933); V.M. Coppleson (195...,http://sharkattackfile.net/spreadsheets/pdf_di...


## Checking dates